In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

from simulation_engine.scenarios.iv.binary_iv import BinaryIV
from simulation_engine.scenarios.iv.continuous_iv import ContinuousIV
from simulation_engine.scenarios.iv.base_iv import IVScenario


In [2]:
# Instantiate and generate data
binary_iv = BinaryIV("IV Dag", 'mypickle')

binary_iv.bound_ate_causaloptim()

loaded Pickle: mypickle
running bound_ate_causaloptim
Data used for ATE calculation: loaded Pickle: mypickle


(0.2, 0.8)

In [8]:
continous_iv = ContinuousIV("IV Dag", 'mycontinuouspickle', 30)

# continous_iv.bound_ate_causaloptim()

Binning continuous data with bin size: 30
loaded binary Data from Pickle: mycontinuouspickle
